In [221]:
#pip install opencv-python


In [222]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import numpy as np


In [223]:
#This is a test cell. Ideally this would be turned into a function
#Photo = cv2.imread('TempFiles/Aaron_Eckhart/Aaron_Eckhart_0001.jpg', 0)
#Photo = cv2.medianBlur(Photo, 5)
#threshold3 = cv2.adaptiveThreshold(Photo, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                            cv2.THRESH_BINARY, 11, 2)


In [224]:
#image we are modifying
#plt.subplot(2, 2, 1), plt.imshow(Photo, 'gray')
#plt.title('Original')
#plt.xticks([]), plt.yticks([])
#plt.show()
#Gaussian Threshold
#plt.subplot(2, 2, 4), plt.imshow(threshold3, 'gray')
#plt.title('Threshold3')
#plt.xticks([]), plt.yticks([])
#plt.show()


In [225]:
from PIL import Image

photosDirectory = []

#First we load all of our files. 
directory = 'SmallerDataSet(cleaned)/'
for file2 in os.listdir(directory):
    path = directory + file2 
    pathForTemp = "TempFiles/" + file2
    try:
        os.makedirs(pathForTemp)
    except:
        VOID = "do nothing :)"
        #This is just to skip the file already exists error
    for FilepathPhotos in os.listdir(path):
        pathForPhoto = path + "/" + FilepathPhotos
        photosDirectory.append(pathForPhoto)
        #Dont run this code it opens every single image. (Good to see what images are being used)
        #image = Image.open(pathForPhoto) 
        #image.show()
#photosDirectory stores the file path for each image we are gonna use
#photosDirectory

In [226]:
#Now we make a function that finds the faces of each picture
FaceDat = cv2.CascadeClassifier("TrainingData/haarcascade_frontalface_default.xml")
EyeDat = cv2.CascadeClassifier("TrainingData/haarcascade_eye.xml")
def faceDetect(path, newDir):
    Photo = cv2.imread(path)
    
    #Blur to remove extra noise
    Photo2 = cv2.medianBlur(Photo, 3) 
    grysc = cv2.cvtColor(Photo2, cv2.COLOR_BGR2GRAY)
    faceDetected = FaceDat.detectMultiScale(grysc, 1.1, 4)
    eyeDetected = EyeDat.detectMultiScale(grysc, 1.3, 4)
    #This area ensures only one face is found per image
    facialPadding = 20
    #The way the algorithm works is it puts faces its less confident in first and most confident last so if multiple pictures are found we just pick the last one.
    #Should work most of the time
    faceFound = False
    if(len(faceDetected) != 1):

        #Gets eye coordinate
        count = 0
        eye1xCord = 0
        eye1ycord = 0
        eye2xCord = 0
        eye2ycord = 0
        for(ex, ey, ew, eh) in eyeDetected:
            #Ensures only two eyes are added to the image
            if(count < 2):
                
                #cv2.rectangle(Photo, ((ex), (ey)),((ex+ew), (ey+eh)), (0, 0, 255), 2)
                cv2.imwrite(newDir, Photo, [cv2.IMWRITE_JPEG_QUALITY, 100])
                if(count == 0):
                    eye1xCord = ex
                    eye1ycord = ey
                if(count == 1):
                    eye2xCord = ex
                    eye2ycord = ey
                count = count + 1
        
        numFaces = len(faceDetected)
        
        for i in range(numFaces):
            x1, y1, w1, h1 = faceDetected[i]
            xrang = x1 + w1
            yrang = y1 + h1

            if(eye1xCord, eye2xCord != 0):
                eye1Exist = True
            if(eye2xCord, eye2ycord != 0):
                eye2Exist = True
            #if eyes arent found then we shouldn't put the face because it could lead to errors in our mean faces.
            if(eye1Exist, eye2Exist == True):
                if(faceFound == False):
                    if(eye1xCord > x1):
                        if(eye1xCord < xrang):  
                            if(eye1ycord > y1):
                                if(eye1ycord < yrang):
                                    faceFound = True
                                    #For our paper we can use these rectangles to visualize what we are doing
                                    #cv2.rectangle(Photo, ((x1 - facialPadding), (y1 - facialPadding)),((x1+w1+facialPadding), (y1+h1+facialPadding)), (134, 255, 20), 2)
                                    faceModified = Photo[y1-facialPadding:y1 +facialPadding+h1, x1-facialPadding:x1+facialPadding+w1]
                                    cv2.imwrite(newDir, faceModified, [cv2.IMWRITE_JPEG_QUALITY, 100])

    else:
        (x, y, w, h) = faceDetected[0]
        #cv2.rectangle(Photo, ((x - facialPadding), (y - facialPadding)), ((x+w+facialPadding),(y+h+facialPadding)), (255, 255, 20), 2)
        faceFound = True
        faceModified = Photo[y-facialPadding:y+facialPadding+h,x-facialPadding:x+facialPadding+w]
        cv2.imwrite(newDir, faceModified)
        
    if(faceFound == False):
        #Delete faces that couldn't be centered
        try:
            os.remove(newDir)
        except:
            alreadyRemoved = 0


In [227]:
#This recursively draws a rectangle around all of the faces we are reading.
for photos in photosDirectory:
    newDir = photos.replace("SmallerDataSet(cleaned)", "TempFiles")
    #print(newDir)
    faceDetect(photos, newDir)
